### VectorBT Pro - Aligning MTF time series Data with Resampling

Reference: https://qubitquants.pro/aligning-mtf-data/index.html

In [3]:
import vectorbt as vbt

In [4]:
cols = ["Open","High","Low","Close","Volume"]
m1_data= vbt.YFData.download("BTC-USD", missing_index=True, interval="1m").get(cols)
print(m1_data)

                                   Open          High           Low  \
Datetime                                                              
2023-08-02 15:23:00+00:00  29238.886719  29238.886719  29238.886719   
2023-08-02 15:24:00+00:00  29253.746094  29253.746094  29253.746094   
2023-08-02 15:25:00+00:00  29269.414062  29269.414062  29269.414062   
2023-08-02 15:26:00+00:00  29267.431641  29267.431641  29267.431641   
2023-08-02 15:27:00+00:00  29283.207031  29283.207031  29283.207031   
...                                 ...           ...           ...   
2023-08-09 15:15:00+00:00  29716.365234  29716.365234  29716.365234   
2023-08-09 15:16:00+00:00  29726.031250  29726.031250  29726.031250   
2023-08-09 15:17:00+00:00  29730.318359  29730.318359  29730.318359   
2023-08-09 15:18:00+00:00  29739.048828  29739.048828  29739.048828   
2023-08-09 15:19:00+00:00  29732.863281  29732.863281  29732.863281   

                                  Close    Volume  
Datetime                

In [5]:
# resample 1 min data to different timeframe
m15_data = m1_data.resample("15T").last()
h1_data = m1_data.resample("1h").last()
h4_data = m1_data.resample("4h").last()

In [6]:
# m15 close
m15_close = m15_data.Close

# h4 data
h4_open = h4_data.Open
h4_close = h4_data.Close
h4_high = h4_data.High
h4_low = h4_data.Low

## h1 data
h1_open  = h1_data.Open
h1_close = h1_data.Close
h1_high  = h1_data.High
h1_low   = h1_data.Low

print("================ 15 min close ==================")
print(m15_close)

================ 15 min close ==================
Datetime
2023-08-02 15:15:00+00:00    29296.445312
2023-08-02 15:30:00+00:00    29317.310547
2023-08-02 15:45:00+00:00    29314.318359
2023-08-02 16:00:00+00:00    29288.859375
2023-08-02 16:15:00+00:00    29253.548828
                                 ...     
2023-08-09 14:15:00+00:00    29855.462891
2023-08-09 14:30:00+00:00    29891.080078
2023-08-09 14:45:00+00:00    29765.621094
2023-08-09 15:00:00+00:00    29730.675781
2023-08-09 15:15:00+00:00    29732.863281
Freq: 15T, Name: Close, Length: 673, dtype: float64


In [7]:
rsi_period = 21

# 15m indicators
m15_rsi = vbt.talib("RSI").run(m15_close,[rsi_period]).real.ffill()
m15_bbands = vbt.talib("BBANDS").run(m15_close)
m15_bbands_rsi = vbt.talib("BBANDS").run(m15_rsi)

# h4 indicators
h4_rsi = vbt.talib("RSI").run(h4_close).real.ffill()
h4_bbands = vbt.talib("BBANDS").run(h4_close)
h4_bbands_rsi = vbt.talib("BBANDS").run(h4_rsi)

In [19]:
rsi_period = 21

rsi = vbt.talib("RSI", rsi_window=rsi_period).run(
    m15_data.get("Close"), 
    ["15T", "1H" , "4H"], 
    #window=rsi_period,
    broadcast_kwargs=dict(wrapper_kwargs=dict(freq="15T"))
).real

bbands_price = vbt.talib("BBANDS").run(
    m15_data.get("Close"), 
    ["15T", "1H", "4H"], 
    broadcast_kwargs=dict(wrapper_kwargs=dict(freq="15T"))
)

bbands_rsi = vbt.talib("BBANDS").run(
    rsi,
    timeframe=vbt.Default(["15T", "1H" ,"4H"]),
    skipna=True,
    per_column=True,
    broadcast_kwargs=dict(wrapper_kwargs=dict(freq="15T"))
)

TypeError: RSI() got an unexpected keyword argument 'rsi_window'

In [8]:
## Initialize  dictionary
data = {}

col_values = [
    m15_close, m15_rsi, m15_bbands.upperband, m15_bbands.middleband, m15_bbands.lowerband, 
    m15_bbands_rsi.upperband, m15_bbands_rsi.middleband, m15_bbands_rsi.lowerband
    ]

col_keys = [
    "m15_close", "m15_rsi", "m15_bband_price_upper",  "m15_bband_price_middle", "m15_bband_price_lower", 
    "m15_bband_rsi_upper",  "m15_bband_rsi_middle", "m15_bband_rsi_lower"
         ]

# Assign key, value pairs for method of time series data to store in data dict
for key, time_series in zip(col_keys, col_values):
    data[key] = time_series.ffill()

In [9]:
## Initialize  dictionary
data = {}

## Assign key, value pairs for method 2 of Automated One-liner MTF indicator creation method
col_values = [
    [m15_close.ffill(), rsi['15T'], bbands_price['15T'].upperband, bbands_price['15T'].middleband, bbands_price['15T'].lowerband, bbands_rsi['15T'].upperband, bbands_rsi['15T'].middleband, bbands_rsi['15T'].lowerband],
    [rsi['1H'], bbands_price['1H'].upperband, bbands_price['1H'].middleband, bbands_price['1H'].lowerband, bbands_rsi['1H'].upperband, bbands_rsi['1H'].middleband, bbands_rsi['1H'].lowerband],
    [rsi['4H'], bbands_price['4H'].upperband, bbands_price['4H'].middleband, bbands_price['4H'].lowerband, bbands_rsi['4H'].upperband, bbands_rsi['4H'].middleband, bbands_rsi['4H'].lowerband]
    ]

col_keys = [
    ["m15_close", "m15_rsi", "m15_bband_price_upper",  "m15_bband_price_middle", "m15_bband_price_lower",  "m15_bband_rsi_upper",  "m15_bband_rsi_middle", "m15_bband_rsi_lower"], 
    ["h1_rsi", "h1_bband_price_upper",  "h1_bband_price_middle",  "h1_bband_price_lower",  "h1_bband_rsi_upper",  "h1_bband_rsi_middle", "h1_bband_rsi_lower"],
    ["h4_rsi", "h4_bband_price_upper",  "h4_bband_price_middle",  "h4_bband_price_lower",  "h4_bband_rsi_upper",  "h4_bband_rsi_middle", "h4_bband_rsi_lower" ],
         ]

## Assign key, value pairs for method 2 of Automated One-liner MTF indicator creation method
for lst_series, lst_keys in zip(col_values, col_keys):
    for key, time_series in zip(lst_keys, lst_series):
        data[key] = time_series


NameError: name 'rsi' is not defined